In [1]:
import json
import wikipediaapi
from graph_construct import wiki_spacy_extract_chunk
from script_neo4j import kw_graph_construct
import re

In [2]:
film_list = ["Dune: Part Two","Spider-Man: Far From Home","Uncharted (film)","The Great Gatsby (2013 film)","Don't Look Up","Inception","Harry Potter and the Goblet of Fire (film)","Harry Potter and the Philosopher's Stone (film)","Oppenheimer (film)","Interstellar (film)"]

In [3]:
def get_wiki_content(page_title):
  user_agent = "My Wikipedia Scraper (contact@example.com)"

  wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent)

  page = wiki_wiki.page(page_title)

  if page.exists():
      return page.text
  else:
      return None

In [4]:
films_content = []
for film in film_list:
  films_content.append(get_wiki_content(film))

In [ ]:
def nettoyer_chaine(chaine):
    # Utilisation d'une expression régulière pour ne garder que les caractères alphanumériques et les espaces
    chaine_propre = re.sub(r'[^a-zA-Z0-9\s]', '', chaine)
    # Supprimer les doubles espaces
    chaine_propre = re.sub(r'\s+', ' ', chaine_propre)
    # Retirer les espaces en début et fin de chaîne
    chaine_propre = chaine_propre.strip()
    return chaine_propre

dataset = {}
dataset['title_chunks'] = []

for i, contenu in enumerate(films_content):
  chunks = contenu.split('\n\n')
  title = film_list[i]
  for chunk in chunks:
    chunk = nettoyer_chaine(chunk)
    chunk = chunk.encode('utf-8').decode('utf-8')
    dataset['title_chunks'].append((title, chunk))

In [7]:
test_docs = wiki_spacy_extract_chunk(dataset)
with open('chunk.json', 'w') as file:
    json.dump(test_docs, file)

100%|██████████| 192/192 [00:16<00:00, 11.76it/s]


In [2]:
# Specify the path to your JSON file
json_file_path = "./chunk.json"
# Read the JSON file and load its content into a Python dictionary
with open(json_file_path, "r") as json_file:
    data_dictionary = json.load(json_file)
kw_graph_construct("script.txt", data_dictionary)

In [ ]:
from tqdm import tqdm
from neo4j import GraphDatabase
path = './script.txt'
uri = ""
user = ""
password = ""
driver = GraphDatabase.driver(uri, auth=(user, password))
with driver.session() as session:
  with open(path, "r") as file:
      contenu = file.read()
      requetes = contenu.split('\n')
      for i in tqdm(range(len(requetes))):
        session.run(requetes[i])